In [7]:
import pandas as pd
data = pd.read_csv('./dscan_result/data/sim_hash_result.csv')
print(data.head())

dict ={}
        
for i in range(len(data['label'])):
    if data['label'][i] not in dict:
        dict[data['label'][i]] = len(dict)
print(dict)

data.label.value_counts()

                                             feature  label
0  400 The plain HTTP request was sent to HTTPS p...  nginx
1  HTTP/1.1 400 Bad Request Content-Type: text/ht...    iis
2  HTTP/1.1 400 Bad Request Server: nginx/1.21.4 ...  nginx
3  HTTP/1.1 400 Bad Request Server: nginx/1.23.1 ...  nginx
4  401 Authorization Required 401 Authorization R...  nginx
{'nginx': 0, 'iis': 1, 'lighttpd': 2, 'boa': 3, 'tomcat': 4, 'rompager': 5, 'http server': 6, 'micro httpd': 7, 'jetty': 8, 'tengine': 9}


label
http server    14438
nginx          12569
iis             3582
jetty            951
lighttpd         630
tomcat           331
tengine          260
boa              248
micro httpd      232
rompager         121
Name: count, dtype: int64

In [8]:

for i in range(len(data['label'])):
    data['label'][i] = dict[data['label'][i]]

for i in range(len(data['feature'])):
    data['feature'][i] = str(data['feature'][i]).lstrip().rstrip()
print(data.head(10))

                                             feature label
0  400 The plain HTTP request was sent to HTTPS p...     0
1  HTTP/1.1 400 Bad Request Content-Type: text/ht...     1
2  HTTP/1.1 400 Bad Request Server: nginx/1.21.4 ...     0
3  HTTP/1.1 400 Bad Request Server: nginx/1.23.1 ...     0
4  401 Authorization Required 401 Authorization R...     0
5  400 The plain HTTP request was sent to HTTPS p...     0
6  HTTP/1.0 400 Bad Request Content-Type: text/ht...     2
7  HTTP/1.1 400 Bad Request Content-Type: text/ht...     0
8  400 The plain HTTP request was sent to HTTPS p...     0
9  404 Not Found 404 Not Found nginx/1.20.2 HTTP/...     0


In [9]:
# with open('./dscan_result/data/to_train.csv', 'w', encoding='utf-8') as f:
#     for i in range(len(data['label'])):
#         line = data['feature'][i] + ' , ' + str(data['label'][i])
#         f.write(line)
#         f.write('\n')
#     f.close()

In [10]:
def simpleSampling(group, frac):
    return group.sample(frac=frac)

train_df = data.groupby(data.label).apply(simpleSampling, 0.5).sample(frac=1, ignore_index = True)
print(train_df)

                                                 feature label
0      Not Found 404 Not Found HTTP/1.1 505 HTTP Vers...     0
1      HTTP/1.1 400 Bad Request Server: nginx/1.22.1 ...     0
2      HTTP/1.1 400 Bad Request Data: DATE Server: Ap...     6
3      HTTP/1.1 400 Bad Request Content-Type: text/ht...     1
4      Página não encontrada – O Antagonista Tamanho ...     0
...                                                  ...   ...
16676  HTTP/1.1 400 Bad Request Server: nginx/1.23.3 ...     0
16677  Design life page Not found . Page Not Found . ...     0
16678  HTTP/1.1 505 HTTP Version Not Supported Server...     1
16679  HTTP/1.1 400 Bad Request Content-Type: text/ht...     1
16680  HTTP/1.1 200 OK Data: DATE Server: Apache/2.2....     6

[16681 rows x 2 columns]


In [12]:
data = pd.concat([data, train_df.sample(frac=1, ignore_index=True)]).drop_duplicates(keep=False)
# data = data.append(train_df.sample(frac=1, ignore_index=True)).drop_duplicates(keep = False)
print(data)

                                                 feature label
2      HTTP/1.1 400 Bad Request Server: nginx/1.21.4 ...     0
3      HTTP/1.1 400 Bad Request Server: nginx/1.23.1 ...     0
4      401 Authorization Required 401 Authorization R...     0
5      400 The plain HTTP request was sent to HTTPS p...     0
6      HTTP/1.0 400 Bad Request Content-Type: text/ht...     2
...                                                  ...   ...
33355  HTTP/1.1 400 Bad Request Content-Type: text/ht...     1
33356  HTTP/1.0 400 Bad Request Data: DATE Server: Bo...     3
33357  HTTP/1.1 400 Bad Request Content-Type: text/ht...     1
33360  HTTP/1.0 400 Bad Request Content-Type: text/ht...     2
33361  HTTP/1.0 400 Bad Request Content-Type: text/ht...     2

[16647 rows x 2 columns]


In [14]:
val_df = data.groupby(data.label).apply(simpleSampling, 0.5).sample(frac=1, ignore_index = True)
# test_df = data.append(val_df).drop_duplicates(keep=False)
test_df = pd.concat([data, val_df]).drop_duplicates(keep=False)
train_df.to_csv('./dscan_result/data/train.csv', header=None, index=None)
val_df.to_csv('./dscan_result/data/val.csv', header=None, index=None)
test_df.to_csv('./dscan_result/data/test.csv', header=None, index=None)